In [1]:
from IPython.display import HTML
HTML('''<h2>IMT 575: Sagemaker trial model</h2>
<b><pre>
    Authors: 
    Aftab Alam
    </pre>
</b> 
<p>Date/Time: <span id="datetime"></span></p><script>var dt = new Date();
document.getElementById("datetime").innerHTML=dt.toLocaleString();</script> </p>''')

In [2]:
# enable flag to how all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
! conda install -y -c conda-forge ipywidgets

!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision

Solving environment: done

# All requested packages already installed.



In [4]:

import time

import boto3
import numpy as np
import pandas as pd
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt

import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker import get_execution_role
from sagemaker.session import Session
from sagemaker.analytics import ExperimentAnalytics

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

Let's start by specifying:

The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [5]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-abalone-default'
# customize to your bucket where you have stored the data
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region, bucket)


Fetching the dataset
Following methods split the data into train/test/validation datasets and upload files to S3.

In [6]:
%%time

import io
import boto3
import random

def data_split(FILE_DATA, FILE_TRAIN, FILE_VALIDATION, FILE_TEST, PERCENT_TRAIN, PERCENT_VALIDATION, PERCENT_TEST):
    data = [l for l in open(FILE_DATA, 'r')]
    train_file = open(FILE_TRAIN, 'w')
    valid_file = open(FILE_VALIDATION, 'w')
    tests_file = open(FILE_TEST, 'w')

    num_of_data = len(data)
    num_train = int((PERCENT_TRAIN/100.0)*num_of_data)
    num_valid = int((PERCENT_VALIDATION/100.0)*num_of_data)
    num_tests = int((PERCENT_TEST/100.0)*num_of_data)

    data_fractions = [num_train, num_valid, num_tests]
    split_data = [[],[],[]]

    rand_data_ind = 0

    for split_ind, fraction in enumerate(data_fractions):
        for i in range(fraction):
            rand_data_ind = random.randint(0, len(data)-1)
            split_data[split_ind].append(data[rand_data_ind])
            data.pop(rand_data_ind)

    for l in split_data[0]:
        train_file.write(l)

    for l in split_data[1]:
        valid_file.write(l)

    for l in split_data[2]:
        tests_file.write(l)

    train_file.close()
    valid_file.close()
    tests_file.close()

def write_to_s3(fobj, bucket, key):
    return boto3.Session(region_name=region).resource('s3').Bucket(bucket).Object(key).upload_fileobj(fobj)

def upload_to_s3(bucket, channel, filename):
    fobj=open(filename, 'rb')
    key = prefix+'/'+channel
    url = 's3://{}/{}/{}'.format(bucket, key, filename)
    print('Writing to {}'.format(url))
    write_to_s3(fobj, bucket, key)

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 11.9 µs



## Data ingestion
Next, we read the dataset from the existing repository into memory, for preprocessing prior to training. This processing could be done in situ by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

In [7]:
FILE_DATA = 'domainsDataSet'
s3 = boto3.client('s3')
s3.download_file(bucket, 'domainsDataSet.csv', FILE_DATA)




In [8]:
# Load the dataset
df_domains = pd.read_csv(FILE_DATA)

## append data set
dummy_domain = {'abcdefghijklmnopqrstuvwxyz1234567890-':'dga'}

dummy_record = pd.DataFrame(dummy_domain.items(), columns=['domain', 'domain_type'])
df_domains = pd.concat((df_domains,dummy_record), axis=0)
df_domains.reset_index(drop=True,inplace=True)
df_domains.tail()

,domain,domain_type
1275048,amiami.com,benign
1275049,freedirectorywebsites.com,benign
1275050,ghaninia.ir,benign
1275051,gndoqarrd.dj,dga
1275052,abcdefghijklmnopqrstuvwxyz1234567890-,dga


In [9]:
! pip install tldextract

In [10]:
import tldextract

def extract_domain_subdomain(record):
    domain = record.domain
    ret=''
    try:
        ext = tldextract.extract(domain)
        ret = ext.domain
    except :
        print(record)
    return ret
def get_y(row):
    if row.domain_type.lower()=='dga':
        return 1
    elif row.domain_type.lower()=='benign':
        return 0
    else :
        return 1
    
df_domains.loc[:,'domain_subdomain'] = df_domains.apply(lambda row : extract_domain_subdomain(row), axis=1 )
## 1 for dga and 0 for benign
df_domains.loc[:,'Y'] = df_domains.apply(lambda row : get_y(row), axis=1 )

domain         NaN
domain_type    dga
Name: 308868, dtype: object
domain            NaN
domain_type    benign
Name: 940873, dtype: object


In [11]:
df_domains.head()

,domain,domain_type,domain_subdomain,Y
0,sugaprof.ru,benign,sugaprof,0
1,zastudents.com,benign,zastudents,0
2,www.youvezmcohdi.ml,dga,youvezmcohdi,1
3,izmirgundemi.net,benign,izmirgundemi,0
4,benchlife.info,benign,benchlife,0


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = None
X1=None
# adding paramter in TfidfVectorizer to using tf_idf tranformation and limiting max feature to 10000
vectorizer = TfidfVectorizer(strip_accents='ascii',use_idf=True,
                             sublinear_tf=True,smooth_idf=True,
                             binary=False,analyzer='char_wb',
                            max_features = 100,)
X1 = vectorizer.fit_transform(df_domains.domain_subdomain).toarray()
X1.shape

(1275053, 40)

In [13]:
df_X = pd.DataFrame(data=X1,columns=vectorizer.get_feature_names(),index=df_domains.index)

df_X.head()

,,-,.,0,1,2,3,4,5,6,...,q,r,s,t,u,v,w,x,y,z
0,0.268710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.304522,0.304733,0.000000,0.364446,0.000000,0.0,0.0,0.000000,0.000000
1,0.233469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.448291,0.450999,0.316649,0.000000,0.0,0.0,0.000000,0.391907
2,0.207546,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.281490,0.321072,0.0,0.0,0.310465,0.348392
3,0.205414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.232790,0.000000,0.000000,0.278598,0.000000,0.0,0.0,0.000000,0.344813
4,0.246954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [14]:
# Join X1 and Y and save is as FILE DATA 

df_X['Y'] = df_domains['Y']
df_X.to_csv(FILE_DATA, encoding='utf-8', index=False)

In [ ]:
%%time
# SciKit Learn implements several Machine Learning algorithms
import sklearn
print( "Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os

import scipy.stats as stats
import statsmodels.formula.api as smf

In [16]:



#split the downloaded data into train/test/validation files
FILE_DATA = 'domainsDataSet'
FILE_TRAIN = 'domainsDataSet.train'
FILE_VALIDATION = 'domainsDataSet.validation'
FILE_TEST = 'domainsDataSet.test'
PERCENT_TRAIN = 70
PERCENT_VALIDATION = 15
PERCENT_TEST = 15
data_split(FILE_DATA, FILE_TRAIN, FILE_VALIDATION, FILE_TEST, PERCENT_TRAIN, PERCENT_VALIDATION, PERCENT_TEST)

#upload the files to the S3 bucket
upload_to_s3(bucket, 'train', FILE_TRAIN)
upload_to_s3(bucket, 'validation', FILE_VALIDATION)
upload_to_s3(bucket, 'test', FILE_TEST)

Writing to s3://sagemaker-us-west-2-099176660580/sagemaker/DEMO-xgboost-abalone-default/train/domainsDataSet.train
Writing to s3://sagemaker-us-west-2-099176660580/sagemaker/DEMO-xgboost-abalone-default/validation/domainsDataSet.validation
Writing to s3://sagemaker-us-west-2-099176660580/sagemaker/DEMO-xgboost-abalone-default/test/domainsDataSet.test


## Training the XGBoost model¶
After setting training parameters, we kick off training, and poll for status until training is completed

In [17]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, 'xgboost', '0.90-1')



In [20]:
%%time
import boto3
from time import gmtime, strftime

job_name = 'DEMO-xgboost-regression-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

#Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.

create_training_params = \
{
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": bucket_path + "/" + prefix + "/single-xgboost"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.m5.large",
        "VolumeSizeInGB": 5
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "silent":"0",
        "objective":"reg:linear",
        "num_round":"50"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 3600
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + "/" + prefix + '/train',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": bucket_path + "/" + prefix + '/validation',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        }
    ]
}


client = boto3.client('sagemaker', region_name=region)
client.create_training_job(**create_training_params)

import time

status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)
while status !='Completed' and status!='Failed':
    time.sleep(60)
    status = client.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print(status)

Training job DEMO-xgboost-regression-2020-05-06-21-28-07
InProgress
InProgress
InProgress
Failed
CPU times: user 66.8 ms, sys: 3.99 ms, total: 70.7 ms
Wall time: 3min


## Import model into hosting
Register the model with hosting. This allows the flexibility of importing models trained elsewhere.

In [ ]:
%%time
import boto3
from time import gmtime, strftime

model_name=job_name + '-model'
print(model_name)

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_data
}

create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])